In [1]:
from __future__ import print_function
import openpathsampling as paths

# New Network Input in OpenPathSampling

This notebook includes some demonstrations of the features that were added by the E-CAM module that created an improved approach to network creation in OpenPathSampling. The improved approach has two main subunits:

* Improved treatment of interface sets
* Improved treatment of multiple state outer interface

To show some examples, I will create a simple collective variable to use as the basis of tests.

In [2]:
cv = paths.CoordinateFunctionCV("x", lambda snap: snap.xyz[0][0])

## Interface sets

There are a few points to consider about the definition of interface sets. The most common case is that each interface volume is defined by the same CV, with different maximum values of the CV, but the same minimum value. However, it is also possible to have interface sets where both minimum and maximum value are changing (the value of $\lambda$ is not uniquely defined by inspecting the `Volume` objects) or where the interfaces don't all grow in terms of a single common CV.

The latter two examples still aren't really supported by our analysis, but we want to make sure that it is possible to support them in the future. However, it would be helpful to make the most common use cases simple.

### Ways of creating the new InterfaceSet

The most common way to create the interface set is barely changed from the previous version. The only difference that is that we use the `VolumeInterfaceSet` object, instead of the `VolumeFactory.CVRangeVolumeSet` function.

In [3]:
interface_set = paths.VolumeInterfaceSet(cv=cv, minvals=0.0, maxvals=[0.2, 0.3, 0.4])

There are other ways to create the interface set. One approach, which is essentially equivalent, would be:

In [4]:
vol1 = paths.CVDefinedVolume(cv, 0.0, 0.2)
vol2 = paths.CVDefinedVolume(cv, 0.0, 0.3)
vol3 = paths.CVDefinedVolume(cv, 0.0, 0.4)

interface_set_2 = paths.InterfaceSet(volumes=[vol1, vol2, vol3],
                                     cv=cv,
                                     lambdas=[0.2, 0.3, 0.4])

We can also create other interface sets, which don't have unique values of $\lambda$ associated:

In [5]:
interface_set_3 = paths.VolumeInterfaceSet(cv=cv, minvals=[-0.1, -0.2, -0.3], maxvals=[0.1, 0.2, 0.3])

In [6]:
# equivalent to interface_set_3
vol1 = paths.CVDefinedVolume(cv, -0.1, 0.1)
vol2 = paths.CVDefinedVolume(cv, -0.2, 0.2)
vol3 = paths.CVDefinedVolume(cv, -0.3, 0.3)
interface_set_4 = paths.InterfaceSet(volumes=[vol1, vol2, vol3], cv=cv)

We can also force all of the volumes created by a `VolumeInterfaceSet` to be intersected with some fixed volume. Of course, doing the same with an `InterfaceSet` (which takes the interface volumes as input, instead of generating them) is trivial.

In [7]:
cv_y = paths.CoordinateFunctionCV("y", lambda snap: snap.xyz[0][1])
volume_y = paths.CVDefinedVolume(cv_y, -1.0, 1.0)

In [8]:
interface_set_5 = paths.VolumeInterfaceSet(cv=cv, minvals=0.0, maxvals=[0.2, 0.3, 0.4], intersect_with=volume_y)

### Associating values of $\lambda$ with the interfaces

One of the major advances here is that we can access the values of the order parameter associated with each interface. This wasn't possible before, and makes analysis much easier.

In [9]:
print(interface_set.lambdas)

[0.2, 0.3, 0.4]


In [10]:
print(interface_set_2.lambdas)

[0.2, 0.3, 0.4]


In [11]:
print(interface_set_5.lambdas)

[0.2, 0.3, 0.4]


Note that not all interface sets have lambda values associated with them. As mentioned above, in some cases, this is literally impossible.

In [12]:
print(interface_set_3.lambdas)
print(interface_set_4.lambdas)

None
None


### Creating a new interface for the set

Another new feature is the ability to create a new interface for a given set. This only works for `VolumeInterfaceSet`s.

In [13]:
interface_set.new_interface(lambda_i=0.5)

In [14]:
interface_set_5.new_interface(lambda_i=0.5)

In [15]:
# We print the message of the AttributeError, but ignore the crash
try:
    interface_set_2.new_interface(lambda_i=0.5)
except AttributeError as e:
    print(e)

'InterfaceSet' object has no attribute 'new_interface'


### Improving analysis due to the new interface sets

Thanks to these new interface sets, we have the ability to identify the volume of lambda for any volume:

In [16]:
volume = interface_set.volumes[0]
interface_set.get_lambda(volume)

0.2

This makes it possible to perform analyses that were previously guesswork. For example, it used to require a complicated algorithm with several assumptions in order to obtain the value of $\lambda$ for the outermost ensemble. Now it can be obtained with:

In [17]:
outermost_lambda = interface_set.get_lambda(interface_set[-1])
print(outermost_lambda)

0.4


## Multiple State Outer Interface

The other main improvement in this module is the development of a better approach to handle the multiple state outer interface. The MS-outer interface itself is a somewhat confusing object; the previous input setup was even more confusing.

First, let's remember the distinction between an interface and an ensemble. An interface is a volume of points in phase space, whereas an ensemble is a set of rules that define whether a trajectory is in or out of the ensemble. Most ensembles in TIS must begin in a specific initial ensemble. However, the MS-outer ensemble allows the trajectory to begin in any of several states. It is easiest to think of the MS-outer ensemble as the set-theoretic union of MSTIS ensembles associated with several initial states. This means that the "interface" is actually several volumes, which must later be associated with the combined ensemble.

Previously, the MS-outer interface was created by implicitly assuming that certain interfaces given by the user were intended to be turned into an MS-outer interface. This could lead to confusion when one of the user's ensembles "disappeared." Worse, the MS-outer interface was only created under certain specific conditions, leading in inconsistency.

Finally, although the MS-outer ensemble can be used to facilitate replica exchange between different interface sets, the better way of doing that is with a state-swap move. The changes in this module make the MS-outer interface optional, so that it can easily be replaced by the state-swap move.

In [18]:
# add another CV, and interface_set coming from that CV
cv_prime = paths.CoordinateFunctionCV("1-x", lambda snap: 1.0-snap.xyz[0][0])
reverse_set = paths.VolumeInterfaceSet(cv_prime, 0.0, [0.2, 0.3, 0.4])

### New object to represent MS-outer interface

We replace the implicit MS-outer interface with an explicit MS-outer interface. If the user wants an MS-outer, it must be made.

In order to create the correct MS-outer ensemble, the MS-outer interface needs to create a volume for each interface set that it should combine. This means it requires the `InterfaceSet` objects decribed above. One approach to setting up the MS-outer interface is to explicitly create the volumes, and associate them with each interface:

In [19]:
vol = interface_set.new_interface(lambda_i=0.5)
vol_prime = reverse_set.new_interface(lambda_i=0.45)
ms_outer = paths.MSOuterTISInterface(
    interface_sets=[interface_set, reverse_set],
    volumes=[vol, vol_prime],
    lambdas=[0.5, 0.45]
)

Since the `VolumeInterfaceSet` knows how to create its own volumes for a given $\lambda$, there is a simpler way to make the equivalent MS-outer interface:

In [20]:
simple_ms_outer = paths.MSOuterTISInterface.from_lambdas({
    interface_set: 0.5,
    reverse_set: 0.45
})

### Making the MS-outer interface optional

The MS-outer interface object is then given as an option to the `TransitionNetwork` for either MSTIS or MISTIS. The network can be build either with or without the MS-outer interface.

In [21]:
state_A = paths.CVDefinedVolume(cv, 0.0, 0.2)
state_B = paths.CVDefinedVolume(cv_prime, 0.0, 0.2)

# with
network_with_msouter = paths.MSTISNetwork(
    [(state_A, interface_set), (state_B, reverse_set)],
    ms_outers=simple_ms_outer
)


# without
network_without_msouter = paths.MSTISNetwork(
    [(state_A, interface_set), (state_B, reverse_set)]
)

We can see that the the MS-outer ensemble in the MSTIS network provides a way to connect the transitions associated with the two interface sets.

In [22]:
print(network_with_msouter.special_ensembles['ms_outer'])

{<openpathsampling.ensemble.UnionEnsemble object at 0x1117dd350>: [<openpathsampling.high_level.transition.TISTransition object at 0x111783350>, <openpathsampling.high_level.transition.TISTransition object at 0x11175a650>]}


In [23]:
print('ms_outer' in network_without_msouter.special_ensembles)

False
